In [1]:
import sys
!{sys.executable} -m pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:

import pandas as pd
import numpy as np

In [3]:
health = pd.read_csv(r"C:\Users\hi\Desktop\Data Analytics\Data Sets\Old\healthcare_dataset - healthcare_dataset.csv")
health.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.28131,328,Urgent,2024-02-02,Paracetamol,Normal
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.32729,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.09608,205,Emergency,2022-10-07,Aspirin,Normal
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.78241,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.31781,458,Urgent,2022-10-09,Penicillin,Abnormal


In [5]:
health.columns

Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object')

In [6]:
health['Date of Admission'] = pd.to_datetime(health['Date of Admission'])
health['Discharge Date'] = pd.to_datetime(health['Discharge Date'])

In [7]:
health['Days Admitted'] = (health['Discharge Date'] - health['Date of Admission']).dt.days

In [9]:
health['Days Admitted'].unique()

array([ 2,  6, 15, 30, 20,  4, 12, 10, 13,  3, 16, 28, 23,  5, 21, 25,  7,
       19, 29, 24, 27,  9, 14, 26, 17, 22,  8, 18,  1, 11])

In [10]:
health['Cost Intensity'] = health['Billing Amount'] / health['Days Admitted']

In [32]:
X = health.drop(columns=['Name','Date of Admission','Hospital','Insurance Provider','Room Number', 'Discharge Date','Test Results'])
y = health['Test Results']

In [38]:
X.columns

Index(['Age', 'Gender', 'Blood Type', 'Medical Condition', 'Doctor',
       'Billing Amount', 'Admission Type', 'Medication', 'Days Admitted',
       'Cost Intensity'],
      dtype='object')

In [88]:

#from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.dummy import DummyClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler

In [89]:
# selecting categorical and numeric values
numeric_values = X.select_dtypes('int64','float64').columns
categoric_values = X.select_dtypes('object').columns

#column transformers
preprocessing = ColumnTransformer( 
    transformers = [
    ('num', RobustScaler(), numeric_values),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categoric_values)
    ]
)

#model pipeline
model = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('smote', SMOTE(random_state=42)),
        ('classifier', RandomForestClassifier(random_state=42))
    ]
)

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [91]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  Index(['Age', 'Days Admitted'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Gender', 'Blood Type', 'Medical Condition', 'Doctor', 'Admission Type',
       'Medication'],
      dtype='object'))])),
                ('smote', SMOTE(random_state=42)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [92]:
y_prediction = model.predict(X_test)
print('Accuracy score:', accuracy_score(y_test, y_prediction))

Accuracy score: 0.36666666666666664


In [93]:
print('Classification Report:',classification_report(y_test,y_prediction))

Classification Report:               precision    recall  f1-score   support

    Abnormal       0.45      0.42      0.43        24
Inconclusive       0.32      0.32      0.32        19
      Normal       0.32      0.35      0.33        17

    accuracy                           0.37        60
   macro avg       0.36      0.36      0.36        60
weighted avg       0.37      0.37      0.37        60



In [83]:
# #model train
# model.fit(X, y)

In [85]:
#model evaluation
# print('Accuracy score:', accuracy_score(y, y_pred))

In [84]:
# print('Classification Report', classification_report(y, y_pred))

In [58]:
#model.named_steps['preprocessing'].get_feature_names_out()

In [59]:
# y_train.value_counts()

Test Results
Abnormal        67
Normal          60
Inconclusive    53
Name: count, dtype: int64

In [16]:
# #scalling numeric values
# numeric_scalling = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='median')),
#         ('scalling', StandardScaler())
#     ]
# )

In [17]:
# # encoding categorical values
# categoric_encoding = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='constant', missing_values='missing' )),
#         ('encode', OneHotEncoder(handle_unknown='ignore'))
#     ]
# )

In [18]:
# # preprocessing the data for modelling
# preprocessing = ColumnTransformer(
#     transformers=[
#         ('numeric', numeric_scalling, numeric_values),
#         ('categoric', categoric_encoding, categoric_values)
#     ]
# )

In [19]:
# #model pipeline




# from xgboost import XGBClassifier
# model_pipeline = Pipeline(
#     steps=[
#         ('preprocessing', preprocessing ),
#         ('ensembling', XGBClassifier())
#     ]
# )

In [20]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
# # #encoding y column
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_test = le.fit_transform(y_test)

In [22]:
# #training the model
# model_pipeline.fit(X_train, y_train)

In [23]:
# #model evaluation
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# #accuracy score evaluation
# y_pred = model_pipeline.predict(X_test)
# print(accuracy_score(y_test, y_pred))

In [24]:
#print(confusion_matrix(y_test, y_pred))

In [25]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# cm =confusion_matrix(y_test, y_pred)
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,  )
# plt.ylabel('True Labels')
# plt.xlabel('Predicted labels')
# plt.show()